In [25]:
from train import *
args = Args(exp_version="3.0", test=True, batch_size=4)


config = exp_configs.get_experiment_config(
        args.exp_version,
        local=args.test,
        vm_name=args.vm_name,
        batch_size=args.batch_size,
    )

3 0
3 0


In [26]:
fns = build_partial_functions(config)
init_rng = jax.random.PRNGKey(config.random_seed)
cur_state = fns["restore_state_from_checkpoint"](init_rng)

Starting from scratch.
Using Dummy Data

ClimSimulator
-------------------------
inputs(x)                      = bf16[4 490]           
expand_dims(x)                 = bf16[4 490 1]         
concat_x_learn(x)              = f32[4 490 128]        
concat_y_learn(x)              = f32[4 798 128]        
pos_emb(x)                     = f32[4 798 128]        
Inside SelfAttention
 ....  ....  ....  .... 
inputs(x)                      = f32[4 798 128]        
linear(x)                      = f32[4 798 384]        
split(q)                       = f32[4 798 2 64]       
split(k)                       = f32[4 798 2 64]       
split(v)                       = f32[4 798 2 64]       
(attn_logits)                  = f32[4 2 798 798]      
(attn_weights)                 = f32[4 2 798 798]      
(attn)                         = f32[4 798 2 64]       
merge(attn_vec)                = f32[4 798 128]        
out_proj(out)                  = f32[4 798 128]        
Inside MLPBlock
 ....  ....  ....

num_params = 0.600 M

Clip Grads: 1.0
Optimizer: adamw
Optimizer kwargs: {'weight_decay': 0.05}
Optimizer addons: {'clip_grads': 1.0, 'grad_acc_steps': None, 'adaptive_grad_clip': None}


In [27]:
# p = config.denorm_arr_path
# x_denorm_mean = np.load(os.path.join(p, "x_denorm_mean.npy"))
# x_denorm_std = np.load(os.path.join(p, "x_denorm_std.npy"))
# y_denorm_mean = np.load(os.path.join(p, "y_denorm_mean.npy"))
# y_denorm_std = np.load(os.path.join(p, "y_denorm_std.npy"))
# scale_output_weights = np.load(os.path.join(p, "scale_output_weights.npy"))

In [28]:
# import pandas as pd
# from climsim_low_res_train.climsim_low_res_train_dataset_builder import (
#     ABLATED_COL_NAMES,
# )

# sub_df = pd.read_csv(config.data_dir + "/sample_submission.csv", nrows=1)
# sub_df.drop(columns=["sample_id"], inplace=True)
# sub_row = sub_df.iloc[0].values.astype(np.float32)

# # Find out index of the ablated columns
# not_ablated_col_indices = [
#     sub_df.columns.tolist().index(col)
#     for col in sub_df.columns.tolist()
#     if col not in ABLATED_COL_NAMES and col != "sample_id"
# ]

# len(not_ablated_col_indices)

# np.save("not_ablated_col_indices.npy", not_ablated_col_indices)
# t = np.load("not_ablated_col_indices.npy")

In [35]:
train_data, val_data, test_data = fns["build_input"]()

inputs = next(train_data)

net = fns["net"]
train_state = cur_state["train_state"]

net_apply = jax.pmap(
    lambda *a: net.apply(*a, is_training=False),
    axis_name="i",
)

y_pred, _ = net_apply(
    train_state.ema_params,
    train_state.ema_state,
    None,
    inputs,
)

y_pred = y_pred.reshape(-1, y_pred.shape[2])
y_true = inputs["targets"].reshape(-1, y_pred.shape[-1])

dnorm_fns = utils.denorm_and_scale_wrapper(config.denorm_arr_path)
dnorm = dnorm_fns["denorm_output"]
dnorm_scale = dnorm_fns["denorm_and_scale_output"]
dnorm_scale_and_expand = dnorm_fns["denorm_scale_and_expand_output"]

y_pred_dnorm = dnorm(y_pred)
y_true_dnorm = dnorm(inputs["targets"].reshape(-1, y_pred.shape[-1]))

y_pred_old = dnorm_scale(y_pred)
y_true_old = dnorm_scale(y_true)

y_pred_new = dnorm_scale_and_expand(y_pred)
y_true_new = dnorm_scale_and_expand(y_true)

r2_score_dnorm = calc_r2_score(y_true_dnorm, y_pred_dnorm)
r2_score_dnorm = jnp.mean(r2_score_dnorm)

r2_score_old = calc_r2_score(y_true_old, y_pred_old)
r2_score_old = jnp.mean(r2_score_old)

r2_score_new = calc_r2_score(y_true_new, y_pred_new)
r2_score_new = jnp.mean(r2_score_new)

direct_r2 = calc_r2_score(y_true, y_pred)
direct_r2 = jnp.mean(direct_r2)

r2_score_old, r2_score_new, r2_score_dnorm, direct_r2

Using Dummy Data

ClimSimulator
-------------------------
inputs(x)                      = f32[4 490]            
expand_dims(x)                 = f32[4 490 1]          
concat_x_learn(x)              = f32[4 490 128]        
concat_y_learn(x)              = f32[4 798 128]        
pos_emb(x)                     = f32[4 798 128]        
Inside SelfAttention
 ....  ....  ....  .... 
inputs(x)                      = f32[4 798 128]        
linear(x)                      = f32[4 798 384]        
split(q)                       = f32[4 798 2 64]       
split(k)                       = f32[4 798 2 64]       
split(v)                       = f32[4 798 2 64]       
(attn_logits)                  = f32[4 2 798 798]      
(attn_weights)                 = f32[4 2 798 798]      
(attn)                         = f32[4 798 2 64]       
merge(attn_vec)                = f32[4 798 128]        
out_proj(out)                  = f32[4 798 128]        
Inside MLPBlock
 ....  ....  ....  .... 
gelu-dense(x)  

(Array(-0.8052857, dtype=float32),
 Array(-0.80362886, dtype=float32),
 Array(0.13759618, dtype=float32),
 Array(-0.94311, dtype=float32))

In [36]:
from sklearn.metrics import r2_score as sk_r2

b_r2 = []
for b in range(y_pred.shape[0]):
    r2 = sk_r2(y_true[b], y_pred[b])
    b_r2.append(r2)

b_r2, np.mean(b_r2)

([-1.140182697697374,
  -0.998114757917345,
  -0.6102162701829839,
  -1.0239268103195993],
 -0.9431101340293255)

In [37]:
@jax.vmap
def my_r2(y_true, y_pred):
    numerator = jnp.sum((y_true - y_pred) ** 2)
    denominator = jnp.sum((y_true - jnp.mean(y_true)) ** 2)
    r2 = 1 - (numerator / (denominator + 1e-7))
    return r2

b_r2 = my_r2(y_true, y_pred)
b_r2, jnp.mean(b_r2)

(Array([-1.1401825 , -0.9981147 , -0.61021614, -1.0239267 ], dtype=float32),
 Array(-0.94311, dtype=float32))

In [32]:
# v_my_r2 = jax.vmap(my_r2)


(Array([-0.7557789, -1.0223663, -1.0403209, -0.99517  ], dtype=float32),
 Array(-0.95340896, dtype=float32))

In [56]:
def apply_placeholder(y: Array):
    B = y.shape[0]
    placeholder = jnp.zeros((B, 368))
    placeholder = placeholder.at[:, t].set(
        y, indices_are_sorted=True, unique_indices=True
    )
    return placeholder

y_pred, y_true = apply_placeholder(y_pred), apply_placeholder(y_true)

In [57]:
y_pred.shape, y_true.shape

((4, 368), (4, 368))

In [24]:
r2_score = calc_r2_score(y_true, y_pred)
r2_score = jnp.mean(r2_score)

r2_score

Array(-0.9507216, dtype=float32)